In [1]:
pip install transformers datasets torch pandas numpy scikit-learn rouge-score nltk bert-score

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install POT

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [ ]:
import pandas as pd
import numpy as np
import torch
from transformers import T5Tokenizer, T5ForConditionalGeneration, Trainer, TrainingArguments, BertTokenizer, BertModel
from datasets import Dataset
from sklearn.metrics import f1_score
from rouge_score import rouge_scorer
import nltk
from bert_score import score as bert_score
from collections import Counter
import ot  # For WMD optimal transport

nltk.download('wordnet')
nltk.download('omw-1.4')
from nltk.translate.meteor_score import meteor_score

# Try importing moverscore; if not available, we'll approximate
try:
    from moverscore import get_moverscore
    MOVESCORE_AVAILABLE = True
except ImportError:
    MOVESCORE_AVAILABLE = False
    print("MoverScore library not available. Approximating with BERT embeddings and WMD.")

# Set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

def load_data(file_path):
    df = pd.read_csv(file_path)
    return df[['tweet', 'GT Target']]

def preprocess_data(df):
    df['tweet'] = df['tweet'].str.strip().str.lower()
    return df

def to_dataset(df):
    return Dataset.from_pandas(df)

def calculate_kqwr(prediction, reference):
    pred_words = set(prediction.lower().split())
    ref_words = set(reference.lower().split())
    if not pred_words or not ref_words:
        return 0.0
    overlap = len(pred_words & ref_words)
    total_unique = len(pred_words | ref_words)
    return overlap / total_unique if total_unique > 0 else 0.0

def calculate_td_tc(predictions, references):
    td = sum(1 for pred, ref in zip(predictions, references) if pred == ref)
    tc = len(predictions)
    return td / tc if tc > 0 else 0.0

def get_bert_embeddings(texts, tokenizer, model):
    if not texts:  # Handle empty input
        return np.zeros((1, 768))  # Return a dummy embedding for consistency
    inputs = tokenizer(texts, return_tensors="pt", padding=True, truncation=True, max_length=512).to(device)
    with torch.no_grad():
        outputs = model(**inputs)
    return outputs.last_hidden_state.cpu().numpy()  # Shape: (batch_size, seq_len, hidden_size)

def calculate_moverscore(predictions, references, bert_tokenizer, bert_model):
    if MOVESCORE_AVAILABLE:
        scores = [get_moverscore(pred, ref) for pred, ref in zip(predictions, references)]
        return np.mean(scores)
    else:
        scores = []
        for pred, ref in zip(predictions, references):
            pred_emb = get_bert_embeddings([pred], bert_tokenizer, bert_model)[0].mean(axis=0)
            ref_emb = get_bert_embeddings([ref], bert_tokenizer, bert_model)[0].mean(axis=0)
            dist_matrix = np.linalg.norm(pred_emb - ref_emb)
            scores.append(1 / (1 + dist_matrix))
        return np.mean(scores)

def calculate_yisi(predictions, references, bert_tokenizer, bert_model):
    scores = []
    for pred, ref in zip(predictions, references):
        pred_emb = get_bert_embeddings([pred], bert_tokenizer, bert_model)[0].mean(axis=0)
        ref_emb = get_bert_embeddings([ref], bert_tokenizer, bert_model)[0].mean(axis=0)
        cosine_sim = np.dot(pred_emb, ref_emb) / (np.linalg.norm(pred_emb) * np.linalg.norm(ref_emb) + 1e-8)
        precision = recall = cosine_sim
        f_score = 2 * (precision * recall) / (precision + recall + 1e-8)
        scores.append(f_score)
    return np.mean(scores)

def calculate_wmd(predictions, references, bert_tokenizer, bert_model):
    scores = []
    for pred, ref in zip(predictions, references):
        pred_words = pred.lower().split()
        ref_words = ref.lower().split()

        if not pred_words or not ref_words:  # Handle empty cases
            scores.append(0.0)  # Default to 0 if empty
            continue

        pred_embs = get_bert_embeddings(pred_words, bert_tokenizer, bert_model)[0]  # Shape: (n, hidden_size)
        ref_embs = get_bert_embeddings(ref_words, bert_tokenizer, bert_model)[0]  # Shape: (m, hidden_size)

        if pred_embs.shape[0] == 0 or ref_embs.shape[0] == 0:
            scores.append(0.0)  # Default distance for empty cases
            continue

        # Normalized bag-of-words vectors
        pred_counts = np.ones(len(pred_words)) / len(pred_words) if pred_words else np.array([1.0])
        ref_counts = np.ones(len(ref_words)) / len(ref_words) if ref_words else np.array([1.0])

        # Compute distance matrix
        dist_matrix = np.linalg.norm(pred_embs[:, np.newaxis] - ref_embs[np.newaxis, :], axis=2)

        # Ensure dist_matrix shape matches pred_counts and ref_counts
        if dist_matrix.shape != (len(pred_counts), len(ref_counts)):
            scores.append(0.0)  # Fallback for mismatch
            continue

        # Compute WMD using optimal transport
        try:
            wmd_dist = ot.emd2(pred_counts, ref_counts, dist_matrix)
            scores.append(wmd_dist)
        except ValueError as e:
            print(f"WMD calculation failed for pred: '{pred}', ref: '{ref}' - {e}")
            scores.append(0.0)  # Fallback for any OT errors

    return np.mean(scores) if scores else 0.0

def calculate_greedy_matching(predictions, references, bert_tokenizer, bert_model):
    scores = []
    for pred, ref in zip(predictions, references):
        pred_tokens = pred.lower().split()
        ref_tokens = ref.lower().split()

        if not pred_tokens or not ref_tokens:
            scores.append(0.0)
            continue

        pred_embs = get_bert_embeddings(pred_tokens, bert_tokenizer, bert_model)[0]
        ref_embs = get_bert_embeddings(ref_tokens, bert_tokenizer, bert_model)[0]

        g_pr = 0
        for ref_emb in ref_embs:
            cos_sims = [np.dot(ref_emb, pred_emb) / (np.linalg.norm(ref_emb) * np.linalg.norm(pred_emb) + 1e-8)
                        for pred_emb in pred_embs]
            g_pr += max(cos_sims)
        g_pr /= len(ref_tokens)

        g_rp = 0
        for pred_emb in pred_embs:
            cos_sims = [np.dot(pred_emb, ref_emb) / (np.linalg.norm(pred_emb) * np.linalg.norm(ref_emb) + 1e-8)
                        for ref_emb in ref_embs]
            g_rp += max(cos_sims)
        g_rp /= len(pred_tokens)

        gm_score = (g_pr + g_rp) / 2
        scores.append(gm_score)

    return np.mean(scores)

def calculate_metrics(model, tokenizer, dataset, sample_size=100):
    model.eval()
    sample = dataset.select(range(min(sample_size, len(dataset))))
    predictions, references = [], []

    bert_tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
    bert_model = BertModel.from_pretrained('bert-base-uncased').to(device)

    with torch.no_grad():
        for example in sample:
            input_text = example['tweet']
            inputs = tokenizer(input_text, return_tensors="pt", max_length=512, truncation=True).to(device)
            outputs = model.generate(**inputs)
            pred = tokenizer.decode(outputs[0], skip_special_tokens=True).strip()
            predictions.append(pred)
            references.append(example['GT Target'])
            print(f"Predicted: {pred},Ground Truth : {example['GT Target']}")

    f1 = f1_score(references, predictions, average='weighted')
    scorer = rouge_scorer.RougeScorer(['rouge1', 'rougeL'], use_stemmer=True)
    rouge_scores = [scorer.score(ref, pred) for ref, pred in zip(references, predictions)]
    rouge1 = np.mean([s['rouge1'].fmeasure for s in rouge_scores])
    rougeL = np.mean([s['rougeL'].fmeasure for s in rouge_scores])
    meteor_scores = [meteor_score([ref.split()], pred.split()) for ref, pred in zip(references, predictions)]
    meteor = np.mean(meteor_scores)
    P, R, F1 = bert_score(predictions, references, lang="en", verbose=False)
    bertscore_f1 = F1.mean().item()

    kqwr_scores = [calculate_kqwr(pred, ref) for pred, ref in zip(predictions, references)]
    kqwr = np.mean(kqwr_scores)
    td_tc = calculate_td_tc(predictions, references)

    moverscore = calculate_moverscore(predictions, references, bert_tokenizer, bert_model)
    yisi = calculate_yisi(predictions, references, bert_tokenizer, bert_model)
    wmd = calculate_wmd(predictions, references, bert_tokenizer, bert_model)
    greedy_matching = calculate_greedy_matching(predictions, references, bert_tokenizer, bert_model)

    return {
        "F1": f1,
        "ROUGE-1": rouge1,
        "ROUGE-L": rougeL,
        "METEOR": meteor,
        "BERTScore": bertscore_f1,
        "KqWR": kqwr,
        "td/tc": td_tc,
        "MoverScore": moverscore,
        "YiSi": yisi,
        "WMD": wmd,
        "GreedyMatching": greedy_matching
    }

def tokenize_data(example, tokenizer):
    input_text = example['tweet']
    target_text = example['GT Target']
    inputs = tokenizer(input_text, max_length=512, truncation=True, padding="max_length")
    targets = tokenizer(target_text, max_length=128, truncation=True, padding="max_length")
    inputs["labels"] = targets["input_ids"]
    return inputs

def train_model(model, tokenizer, train_dataset):
    training_args = TrainingArguments(
        output_dir="./t5_trained",
        num_train_epochs=3,
        per_device_train_batch_size=8,
        save_steps=500,
        save_total_limit=2,
        logging_steps=100,
    )
    trainer = Trainer(model=model, args=training_args, train_dataset=train_dataset)
    trainer.train()
    return model

def main(file_path):
    df = load_data(file_path)
    df = preprocess_data(df)
    dataset = to_dataset(df)

    tokenizer = T5Tokenizer.from_pretrained("t5-base")
    model = T5ForConditionalGeneration.from_pretrained("t5-base").to(device)

    print("Calculating Pre-training Metrics...")
    pre_metrics = calculate_metrics(model, tokenizer, dataset)
    print("Pre-training Metrics:", pre_metrics)

    train_size = min(1000, len(dataset))
    train_dataset = dataset.select(range(train_size))
    tokenized_train_dataset = train_dataset.map(lambda x: tokenize_data(x, tokenizer), batched=False)

    print("Training T5...")
    trained_model = train_model(model, tokenizer, tokenized_train_dataset)

    print("Calculating Post-training Metrics...")
    post_metrics = calculate_metrics(trained_model, tokenizer, dataset.select(range(1000,1100)))
    print("Post-training Metrics:", post_metrics)

if __name__ == "__main__":
    main("/content/randomized_output.csv")

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


MoverScore library not available. Approximating with BERT embeddings and WMD.
Using device: cuda


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

...

Passing a tuple of `past_key_values` is deprecated and will be removed in Transformers v4.48.0. You should pass an instance of `EncoderDecoderCache` instead, e.g. `past_key_values=EncoderDecoderCache.from_legacy_cache(past_key_values)`.


Step,Training Loss
100,2.412700
200,0.043000
300,0.015900


Calculating Post-training Metrics...
Predicted: climate change is a concern,Ground Truth : climate change is a concern
Predicted: creationism,Ground Truth : creationism
Predicted: climate change is a concern,Ground Truth : climate change is a concern
Predicted: gay rights,Ground Truth : gay rights
Predicted: creationism,Ground Truth : creationism
Predicted: creationism,Ground Truth : gay rights
Predicted: creationism,Ground Truth : creationism
Predicted: gay rights,Ground Truth : gay rights
Predicted: creationism,Ground Truth : creationism
Predicted: gay rights,Ground Truth : gay rights
Predicted: climate change is a concern,Ground Truth : climate change is a concern
Predicted: gay rights,Ground Truth : gay rights
Predicted: climate change is a concern,Ground Truth : climate change is a concern
Predicted: creationism,Ground Truth : creationism
Predicted: climate change is a concern,Ground Truth : climate change is a concern
Predicted: gay rights,Ground Truth : creationism
Predicted: ga

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Post-training Metrics: {'F1': 0.8872108843537414, 'ROUGE-1': np.float64(0.88), 'ROUGE-L': np.float64(0.88), 'METEOR': np.float64(0.7264849999999998), 'BERTScore': 0.9849300980567932, 'KqWR': np.float64(0.88), 'td/tc': 0.88, 'MoverScore': np.float32(0.89404434), 'YiSi': np.float32(0.9630027), 'WMD': np.float64(0.0), 'GreedyMatching': np.float32(1.8807685)}


In [ ]:
### T5 using Vast Dataset

In [6]:
pip install numpy==1.26.4

Defaulting to user installation because normal site-packages is not writeable
   ---------------------------------------- 0.0/15.8 MB ? eta -:--:--
   --- ------------------------------------ 1.6/15.8 MB 8.4 MB/s eta 0:00:02
   ---------- ----------------------------- 4.2/15.8 MB 10.5 MB/s eta 0:00:02
   --------------------- ------------------ 8.4/15.8 MB 14.1 MB/s eta 0:00:01
   ------------------------------------- -- 14.7/15.8 MB 18.1 MB/s eta 0:00:01
   ---------------------------------------- 15.8/15.8 MB 18.1 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 2.1.3
    Uninstalling numpy-2.1.3:
      Successfully uninstalled numpy-2.1.3
Note: you may need to restart the kernel to use updated packages.


  You can safely remove it manually.
  You can safely remove it manually.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
img2dataset 1.45.0 requires albumentations<2,>=1.1.0, but you have albumentations 2.0.5 which is incompatible.
img2dataset 1.45.0 requires pyarrow<16,>=6.0.1, but you have pyarrow 19.0.1 which is incompatible.
tensorflow-intel 2.18.0 requires ml-dtypes<0.5.0,>=0.4.0, but you have ml-dtypes 0.5.1 which is incompatible.
tensorflow-intel 2.18.0 requires tensorboard<2.19,>=2.18, but you have tensorboard 2.19.0 which is incompatible.


In [15]:
!python -m pip install moverscore


Defaulting to user installation because normal site-packages is not writeable


In [25]:
!pip install numpy==1.26.4




Defaulting to user installation because normal site-packages is not writeable
  Using cached numpy-1.26.4-cp310-cp310-win_amd64.whl.metadata (61 kB)
Using cached numpy-1.26.4-cp310-cp310-win_amd64.whl (15.8 MB)
  Attempting uninstall: numpy
    Found existing installation: numpy 1.26.0
    Uninstalling numpy-1.26.0:
      Successfully uninstalled numpy-1.26.0


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
img2dataset 1.45.0 requires albumentations<2,>=1.1.0, but you have albumentations 2.0.5 which is incompatible.
img2dataset 1.45.0 requires pyarrow<16,>=6.0.1, but you have pyarrow 19.0.1 which is incompatible.
tensorflow-intel 2.18.0 requires ml-dtypes<0.5.0,>=0.4.0, but you have ml-dtypes 0.5.1 which is incompatible.
tensorflow-intel 2.18.0 requires tensorboard<2.19,>=2.18, but you have tensorboard 2.19.0 which is incompatible.


In [1]:
import pandas as pd
import numpy as np
import torch
import json
from transformers import T5Tokenizer, T5ForConditionalGeneration, Trainer, TrainingArguments, BertTokenizer, BertModel
from datasets import Dataset
from sklearn.metrics import f1_score
from rouge_score import rouge_scorer
import nltk
from bert_score import score as bert_score
from collections import Counter
import ot

nltk.download('wordnet')
nltk.download('omw-1.4')
from nltk.translate.meteor_score import meteor_score

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

def convert_to_serializable(obj):
    if isinstance(obj, (np.float32, np.float64)):
        return float(obj)
    elif isinstance(obj, np.integer):
        return int(obj)
    elif isinstance(obj, dict):
        return {k: convert_to_serializable(v) for k, v in obj.items()}
    elif isinstance(obj, list):
        return [convert_to_serializable(item) for item in obj]
    else:
        return obj

def save_json(data, file_name):
    serializable_data = convert_to_serializable(data)
    with open(file_name, "w") as f:
        json.dump(serializable_data, f, indent=4)

def save_model(model, tokenizer, output_dir):
    model.save_pretrained(output_dir)
    tokenizer.save_pretrained(output_dir)

def save_predictions_to_csv(predictions, references, filename):
    df = pd.DataFrame({
        'predictions': predictions,
        'ground_truth': references
    })
    df.to_csv(filename, index=False)

def get_bert_embeddings(texts, bert_tokenizer, bert_model):
    inputs = bert_tokenizer(texts, return_tensors="pt", padding=True, truncation=True, max_length=512).to(device)
    with torch.no_grad():
        outputs = bert_model(**inputs)
    return outputs.last_hidden_state.cpu().numpy()

def calculate_yisi(predictions, references, bert_tokenizer, bert_model):
    scores = []
    for pred, ref in zip(predictions, references):
        pred_emb = get_bert_embeddings([pred], bert_tokenizer, bert_model)[0].mean(axis=0)
        ref_emb = get_bert_embeddings([ref], bert_tokenizer, bert_model)[0].mean(axis=0)
        cosine_sim = np.dot(pred_emb, ref_emb) / (np.linalg.norm(pred_emb) * np.linalg.norm(ref_emb) + 1e-8)
        precision = recall = cosine_sim
        f_score = 2 * (precision * recall) / (precision + recall + 1e-8)
        scores.append(f_score)
    return np.mean(scores)

def calculate_moverscore(predictions, references, bert_tokenizer, bert_model):
    scores = []
    for pred, ref in zip(predictions, references):
        pred_emb = get_bert_embeddings([pred], bert_tokenizer, bert_model)[0].mean(axis=0)
        ref_emb = get_bert_embeddings([ref], bert_tokenizer, bert_model)[0].mean(axis=0)
        dist_matrix = np.linalg.norm(pred_emb - ref_emb)
        scores.append(1 / (1 + dist_matrix))
    return np.mean(scores)

def calculate_metrics(model, tokenizer, dataset):
    model.eval()
    predictions, references = [], []

    bert_tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
    bert_model = BertModel.from_pretrained('bert-base-uncased').to(device)

    with torch.no_grad():
        for example in dataset:
            input_text = example['post']
            inputs = tokenizer(input_text, return_tensors="pt", max_length=512, truncation=True).to(device)
            outputs = model.generate(**inputs)
            pred = tokenizer.decode(outputs[0], skip_special_tokens=True).strip()
            predictions.append(pred)
            references.append(example['new_topic'])

    f1 = f1_score(references, predictions, average='weighted')
    scorer = rouge_scorer.RougeScorer(['rouge1', 'rougeL'], use_stemmer=True)
    rouge_scores = [scorer.score(ref, pred) for ref, pred in zip(references, predictions)]
    rouge1 = np.mean([s['rouge1'].fmeasure for s in rouge_scores])
    rougeL = np.mean([s['rougeL'].fmeasure for s in rouge_scores])
    meteor = np.mean([meteor_score([ref.split()], pred.split()) for ref, pred in zip(references, predictions)])
    P, R, F1 = bert_score(predictions, references, lang="en", verbose=False)
    bertscore_f1 = F1.mean().item()
    yisi = calculate_yisi(predictions, references, bert_tokenizer, bert_model)
    moverscore = calculate_moverscore(predictions, references, bert_tokenizer, bert_model)

    metrics = {
        "F1": f1,
        "ROUGE-1": rouge1,
        "ROUGE-L": rougeL,
        "METEOR": meteor,
        "BERTScore": bertscore_f1,
        "YiSi": yisi,
        "MoverScore": moverscore
    }

    return metrics, predictions, references

def tokenize_data(example, tokenizer):
    model_inputs = tokenizer(
        example['post'],
        max_length=512,
        truncation=True,
        padding="max_length"
    )

    with tokenizer.as_target_tokenizer():
        labels = tokenizer(
            example['new_topic'],
            max_length=128,
            truncation=True,
            padding="max_length"
        )

    labels_ids = [
        (label if label != tokenizer.pad_token_id else -100)
        for label in labels["input_ids"]
    ]

    model_inputs["labels"] = labels_ids
    return model_inputs

def train_model(model, tokenizer, train_dataset):
    training_args = TrainingArguments(
        output_dir="./t5_trained",
        num_train_epochs=3,
        per_device_train_batch_size=8,
        save_steps=500,
        save_total_limit=2,
        logging_steps=100,
    )
    trainer = Trainer(model=model, args=training_args, train_dataset=train_dataset)
    trainer.train()
    return model

def main(file_path):
    df = pd.read_csv(file_path)
    df['post'] = df['post'].str.strip().str.lower()
    dataset = Dataset.from_pandas(df)

    tokenizer = T5Tokenizer.from_pretrained("t5-base")
    model = T5ForConditionalGeneration.from_pretrained("t5-base").to(device)

    print("Calculating Pre-training Metrics...")
    pre_metrics, pre_predictions, pre_references = calculate_metrics(model, tokenizer, dataset)
    save_json(pre_metrics, "pre_training_metrics.json")
    save_predictions_to_csv(pre_predictions, pre_references, "pre_training_predictions.csv")
    print("Pre-training Metrics and Predictions saved!")

    print("Tokenizing dataset...")
    tokenized_train_dataset = dataset.map(
        lambda x: tokenize_data(x, tokenizer),
        batched=False,
        remove_columns=dataset.column_names  # Ensures only model inputs remain
    )

    print("Training T5...")
    trained_model = train_model(model, tokenizer, tokenized_train_dataset)

    print("Saving trained model...")
    save_model(trained_model, tokenizer, "./t5_trained_model")

    print("Calculating Post-training Metrics...")
    post_metrics, post_predictions, post_references = calculate_metrics(trained_model, tokenizer, dataset)
    save_json(post_metrics, "post_training_metrics_BART.json")
    save_predictions_to_csv(post_predictions, post_references, "post_training_predictions.csv")
    print("Post-training Metrics and Predictions saved!")

if __name__ == "__main__":
    main("vast_filtered_im.csv")

C:\Users\CSE RGUKT\AppData\Roaming\Python\Python310\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


[nltk_data] Downloading package wordnet to C:\Users\CSE
[nltk_data]     RGUKT\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to C:\Users\CSE
[nltk_data]     RGUKT\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


Using device: cuda


You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


Calculating Pre-training Metrics...


KeyboardInterrupt: 

Defaulting to user installation because normal site-packages is not writeable


#keybart

In [2]:
import pandas as pd
import numpy as np
import torch
import json
from transformers import BartTokenizer, BartForConditionalGeneration, Trainer, TrainingArguments, BertTokenizer, BertModel
from datasets import Dataset
from sklearn.metrics import f1_score
from rouge_score import rouge_scorer
import nltk
from bert_score import score as bert_score
from collections import Counter
import ot

nltk.download('wordnet')
nltk.download('omw-1.4')
from nltk.translate.meteor_score import meteor_score

# Set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Utility functions
def save_json(data, file_name):
    with open(file_name, "w") as f:
        json.dump(data, f, indent=4)

def save_model(model, tokenizer, output_dir):
    model.save_pretrained(output_dir)
    tokenizer.save_pretrained(output_dir)

def save_predictions_to_csv(predictions, references, filename):
    df = pd.DataFrame({
        'predictions': predictions,
        'ground_truth': references
    })
    df.to_csv(filename, index=False)

def get_bert_embeddings(texts, bert_tokenizer, bert_model):
    inputs = bert_tokenizer(texts, return_tensors="pt", padding=True, truncation=True, max_length=512).to(device)
    with torch.no_grad():
        outputs = bert_model(**inputs)
    return outputs.last_hidden_state.cpu().numpy()

def calculate_yisi(predictions, references, bert_tokenizer, bert_model):
    scores = []
    for pred, ref in zip(predictions, references):
        pred_emb = get_bert_embeddings([pred], bert_tokenizer, bert_model)[0].mean(axis=0)
        ref_emb = get_bert_embeddings([ref], bert_tokenizer, bert_model)[0].mean(axis=0)
        cosine_sim = np.dot(pred_emb, ref_emb) / (np.linalg.norm(pred_emb) * np.linalg.norm(ref_emb) + 1e-8)
        precision = recall = cosine_sim
        f_score = 2 * (precision * recall) / (precision + recall + 1e-8)
        scores.append(f_score)
    return np.mean(scores)

def calculate_moverscore(predictions, references, bert_tokenizer, bert_model):
    scores = []
    for pred, ref in zip(predictions, references):
        pred_emb = get_bert_embeddings([pred], bert_tokenizer, bert_model)[0].mean(axis=0)
        ref_emb = get_bert_embeddings([ref], bert_tokenizer, bert_model)[0].mean(axis=0)
        dist_matrix = np.linalg.norm(pred_emb - ref_emb)
        scores.append(1 / (1 + dist_matrix))
    return np.mean(scores)

def calculate_metrics(model, tokenizer, dataset):
    model.eval()
    predictions, references = [], []

    bert_tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
    bert_model = BertModel.from_pretrained('bert-base-uncased').to(device)

    with torch.no_grad():
        for example in dataset:
            input_text = example['post']
            inputs = tokenizer(input_text, return_tensors="pt", max_length=512, truncation=True).to(device)
            outputs = model.generate(**inputs, max_length=128)
            pred = tokenizer.decode(outputs[0], skip_special_tokens=True).strip()
            predictions.append(pred)
            references.append(example['new_topic'])

    # Calculate metrics
    f1 = f1_score(references, predictions, average='weighted')
    scorer = rouge_scorer.RougeScorer(['rouge1', 'rougeL'], use_stemmer=True)
    rouge_scores = [scorer.score(ref, pred) for ref, pred in zip(references, predictions)]
    rouge1 = np.mean([s['rouge1'].fmeasure for s in rouge_scores])
    rougeL = np.mean([s['rougeL'].fmeasure for s in rouge_scores])
    meteor = np.mean([meteor_score([ref.split()], pred.split()) for ref, pred in zip(references, predictions)])
    P, R, F1 = bert_score(predictions, references, lang="en", verbose=False)
    bertscore_f1 = F1.mean().item()

    # Calculate YiSi and MoverScore
    yisi = calculate_yisi(predictions, references, bert_tokenizer, bert_model)
    moverscore = calculate_moverscore(predictions, references, bert_tokenizer, bert_model)

    metrics = {
        "F1": f1,
        "ROUGE-1": rouge1,
        "ROUGE-L": rougeL,
        "METEOR": meteor,
        "BERTScore": bertscore_f1,
        "YiSi": yisi,
        "MoverScore": moverscore
    }

    return metrics, predictions, references

def tokenize_data(example, tokenizer):
    model_inputs = tokenizer(
        example['post'],
        max_length=512,
        truncation=True,
        padding="max_length"
    )

    with tokenizer.as_target_tokenizer():
        labels = tokenizer(
            example['new_topic'],
            max_length=128,
            truncation=True,
            padding="max_length"
        )

    labels_ids = labels["input_ids"]
    labels_ids = [
        (label if label != tokenizer.pad_token_id else -100)
        for label in labels_ids
    ]
    model_inputs["labels"] = labels_ids

    return model_inputs

def train_model(model, tokenizer, train_dataset):
    training_args = TrainingArguments(
        output_dir="./keybart_trained",
        num_train_epochs=3,
        per_device_train_batch_size=8,
        save_steps=500,
        save_total_limit=2,
        logging_steps=100,
    )
    trainer = Trainer(model=model, args=training_args, train_dataset=train_dataset)
    trainer.train()
    return model

def main(file_path):
    df = pd.read_csv(file_path)
    df['post'] = df['post'].str.strip().str.lower()
    dataset = Dataset.from_pandas(df)

    # Load KeyBART explicitly with the correct identifier
    tokenizer = BartTokenizer.from_pretrained("bloomberg/KeyBART")
    model = BartForConditionalGeneration.from_pretrained("bloomberg/KeyBART").to(device)
    print("Loaded KeyBART successfully.")

    # Pre-training evaluation
    print("Calculating Pre-training Metrics...")
    pre_metrics, pre_predictions, pre_references = calculate_metrics(model, tokenizer, dataset)
    save_json(pre_metrics, "pre_training_metrics_KEYBART.json")
    save_predictions_to_csv(pre_predictions, pre_references, "pre_training_predictions_KEYBART.csv")
    print("Pre-training Metrics and Predictions saved!")

    # Tokenize and train
    tokenized_train_dataset = dataset.map(
        lambda x: tokenize_data(x, tokenizer),
        batched=False,
        remove_columns=dataset.column_names
    )
    print("Training KeyBART...")
    trained_model = train_model(model, tokenizer, tokenized_train_dataset)

    # Save trained model
    print("Saving trained model...")
    save_model(trained_model, tokenizer, "./keybart_trained_model")

    # Post-training evaluation
    print("Calculating Post-training Metrics...")
    post_metrics, post_predictions, post_references = calculate_metrics(trained_model, tokenizer, dataset)
    save_json(post_metrics, "post_training_metrics_KEYBART.json")
    save_predictions_to_csv(post_predictions, post_references, "post_training_predictions_KEYBART.csv")
    print("Post-training Metrics and Predictions saved!")

if __name__ == "__main__":
    main("vast_filtered_im.csv")

Error in callback <bound method _WandbInit._resume_backend of <wandb.sdk.wandb_init._WandbInit object at 0x000001B033D763B0>> (for pre_run_cell), with arguments args (<ExecutionInfo object at 1b02addace0, raw_cell="import pandas as pd
import numpy as np
import torc.." store_history=True silent=False shell_futures=True cell_id=vscode-notebook-cell:/c%3A/Users/CSE%20RGUKT/Desktop/Stance/vast_filtered_im/Stance_Updated_2_im.ipynb#X13sZmlsZQ%3D%3D>,),kwargs {}:


TypeError: _WandbInit._resume_backend() takes 1 positional argument but 2 were given

[nltk_data] Downloading package wordnet to C:\Users\CSE
[nltk_data]     RGUKT\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to C:\Users\CSE
[nltk_data]     RGUKT\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


Using device: cuda
Loaded KeyBART successfully.
Calculating Pre-training Metrics...


Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


...

C:\Users\CSE RGUKT\AppData\Roaming\Python\Python310\site-packages\transformers\modeling_utils.py:3353: UserWarning: Moving the following attributes in the config to the generation config: {'early_stopping': True, 'num_beams': 4, 'no_repeat_ngram_size': 3}. You are seeing this warning because you've set generation parameters in the model config, as opposed to in the generation config.
  warnings.warn(


Saving trained model...
Calculating Post-training Metrics...


Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Post-training Metrics and Predictions saved!
Error in callback <bound method _WandbInit._pause_backend of <wandb.sdk.wandb_init._WandbInit object at 0x000001B033D763B0>> (for post_run_cell), with arguments args (<ExecutionResult object at 1b02addabc0, execution_count=2 error_before_exec=None error_in_exec=None info=<ExecutionInfo object at 1b02addace0, raw_cell="import pandas as pd
import numpy as np
import torc.." store_history=True silent=False shell_futures=True cell_id=vscode-notebook-cell:/c%3A/Users/CSE%20RGUKT/Desktop/Stance/vast_filtered_im/Stance_Updated_2_im.ipynb#X13sZmlsZQ%3D%3D> result=None>,),kwargs {}:


TypeError: _WandbInit._pause_backend() takes 1 positional argument but 2 were given

**BART-Base (Bidirectional and Auto-Regressive Transformer)**

In [3]:
import pandas as pd
import numpy as np
import torch
import json
from transformers import BartTokenizer, BartForConditionalGeneration, Trainer, TrainingArguments, BertTokenizer, BertModel
from datasets import Dataset
from sklearn.metrics import f1_score
from rouge_score import rouge_scorer
import nltk
from bert_score import score as bert_score
from collections import Counter
import ot

nltk.download('wordnet')
nltk.download('omw-1.4')
from nltk.translate.meteor_score import meteor_score

# Set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

def save_json(data, file_name):
    with open(file_name, "w") as f:
        json.dump(data, f, indent=4)

def save_model(model, tokenizer, output_dir):
    model.save_pretrained(output_dir)
    tokenizer.save_pretrained(output_dir)

def save_predictions_to_csv(predictions, references, filename):
    df = pd.DataFrame({
        'predictions': predictions,
        'ground_truth': references
    })
    df.to_csv(filename, index=False)

def get_bert_embeddings(texts, bert_tokenizer, bert_model):
    inputs = bert_tokenizer(texts, return_tensors="pt", padding=True, truncation=True, max_length=512).to(device)
    with torch.no_grad():
        outputs = bert_model(**inputs)
    return outputs.last_hidden_state.cpu().numpy()  # Move to CPU for numpy compatibility

def calculate_yisi(predictions, references, bert_tokenizer, bert_model):
    scores = []
    for pred, ref in zip(predictions, references):
        pred_emb = get_bert_embeddings([pred], bert_tokenizer, bert_model)[0].mean(axis=0)
        ref_emb = get_bert_embeddings([ref], bert_tokenizer, bert_model)[0].mean(axis=0)
        cosine_sim = np.dot(pred_emb, ref_emb) / (np.linalg.norm(pred_emb) * np.linalg.norm(ref_emb) + 1e-8)
        precision = recall = cosine_sim
        f_score = 2 * (precision * recall) / (precision + recall + 1e-8)
        scores.append(f_score)
    return np.mean(scores)

def calculate_moverscore(predictions, references, bert_tokenizer, bert_model):
    scores = []
    for pred, ref in zip(predictions, references):
        pred_emb = get_bert_embeddings([pred], bert_tokenizer, bert_model)[0].mean(axis=0)
        ref_emb = get_bert_embeddings([ref], bert_tokenizer, bert_model)[0].mean(axis=0)
        dist_matrix = np.linalg.norm(pred_emb - ref_emb)
        scores.append(1 / (1 + dist_matrix))
    return np.mean(scores)

def calculate_metrics(model, tokenizer, dataset):
    model.eval()
    predictions, references = [], []

    bert_tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
    bert_model = BertModel.from_pretrained('bert-base-uncased').to(device)

    with torch.no_grad():
        for example in dataset:
            input_text = example['post']
            inputs = tokenizer(input_text, return_tensors="pt", max_length=512, truncation=True).to(device)
            outputs = model.generate(**inputs)
            pred = tokenizer.decode(outputs[0], skip_special_tokens=True).strip()
            predictions.append(pred)
            references.append(example['new_topic'])

    # Calculate metrics
    f1 = f1_score(references, predictions, average='weighted')
    scorer = rouge_scorer.RougeScorer(['rouge1', 'rougeL'], use_stemmer=True)
    rouge_scores = [scorer.score(ref, pred) for ref, pred in zip(references, predictions)]
    rouge1 = np.mean([s['rouge1'].fmeasure for s in rouge_scores])
    rougeL = np.mean([s['rougeL'].fmeasure for s in rouge_scores])
    meteor = np.mean([meteor_score([ref.split()], pred.split()) for ref, pred in zip(references, predictions)])
    P, R, F1 = bert_score(predictions, references, lang="en", verbose=False)
    bertscore_f1 = F1.mean().item()

    # Calculate YiSi and MoverScore
    yisi = calculate_yisi(predictions, references, bert_tokenizer, bert_model)
    moverscore = calculate_moverscore(predictions, references, bert_tokenizer, bert_model)

    metrics = {
        "F1": f1,
        "ROUGE-1": rouge1,
        "ROUGE-L": rougeL,
        "METEOR": meteor,
        "BERTScore": bertscore_f1,
        "YiSi": yisi,
        "MoverScore": moverscore
    }

    return metrics, predictions, references

def tokenize_data(example, tokenizer):
    model_inputs = tokenizer(
        example['post'],
        max_length=512,
        truncation=True,
        padding="max_length"
    )

    with tokenizer.as_target_tokenizer():
        labels = tokenizer(
            example['new_topic'],
            max_length=128,
            truncation=True,
            padding="max_length"
        )

    # Replace padding token id's in labels by -100 to ignore in loss
    labels_ids = labels["input_ids"]
    labels_ids = [
        (label if label != tokenizer.pad_token_id else -100)
        for label in labels_ids
    ]
    model_inputs["labels"] = labels_ids

    return model_inputs



def train_model(model, tokenizer, train_dataset):
    training_args = TrainingArguments(
        output_dir="./BART_trained",
        num_train_epochs=3,
        per_device_train_batch_size=8,
        save_steps=500,
        save_total_limit=2,
        logging_steps=100,
    )
    trainer = Trainer(model=model, args=training_args, train_dataset=train_dataset)
    trainer.train()
    return model

def main(file_path):
    df = pd.read_csv(file_path)
    df['post'] = df['post'].str.strip().str.lower()
    dataset = Dataset.from_pandas(df)

    tokenizer = BartTokenizer.from_pretrained("facebook/bart-base")
    model = BartForConditionalGeneration.from_pretrained("facebook/bart-base").to(device)

    # # Pre-training evaluation
    # print("Calculating Pre-training Metrics...")
    # pre_metrics, pre_predictions, pre_references = calculate_metrics(model, tokenizer, dataset)
    # save_json(pre_metrics, "pre_training_metrics_BART.json")
    # save_predictions_to_csv(pre_predictions, pre_references, "pre_training_predictions_BART.csv")
    # print("Pre-training Metrics and Predictions saved!")

    # Tokenize and train
    tokenized_train_dataset = dataset.map(
    lambda x: tokenize_data(x, tokenizer),
    batched=False,
    remove_columns=dataset.column_names
)

    print("Training T5...")
    trained_model = train_model(model, tokenizer, tokenized_train_dataset)

    # Save trained model
    print("Saving trained model...")
    save_model(trained_model, tokenizer, "./BART_trained_model")

    # Post-training evaluation
    print("Calculating Post-training Metrics...")
    post_metrics, post_predictions, post_references = calculate_metrics(trained_model, tokenizer, dataset)
    save_json(post_metrics, "post_training_metrics_BART.json")
    save_predictions_to_csv(post_predictions, post_references, "post_training_predictions_BART.csv")
    print("Post-training Metrics and Predictions saved!")

if __name__ == "__main__":
    main("vast_filtered_im.csv")

Error in callback <bound method _WandbInit._resume_backend of <wandb.sdk.wandb_init._WandbInit object at 0x000001B033D763B0>> (for pre_run_cell), with arguments args (<ExecutionInfo object at 1b033d76c80, raw_cell="import pandas as pd
import numpy as np
import torc.." store_history=True silent=False shell_futures=True cell_id=vscode-notebook-cell:/c%3A/Users/CSE%20RGUKT/Desktop/Stance/vast_filtered_im/Stance_Updated_2_im.ipynb#X15sZmlsZQ%3D%3D>,),kwargs {}:


TypeError: _WandbInit._resume_backend() takes 1 positional argument but 2 were given

[nltk_data] Downloading package wordnet to C:\Users\CSE
[nltk_data]     RGUKT\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to C:\Users\CSE
[nltk_data]     RGUKT\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


Using device: cuda


Map:   0%|          | 0/1980 [00:00<?, ? examples/s]C:\Users\CSE RGUKT\AppData\Roaming\Python\Python310\site-packages\transformers\tokenization_utils_base.py:3980: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(
Map: 100%|██████████| 1980/1980 [00:04<00:00, 434.15 examples/s]


...

C:\Users\CSE RGUKT\AppData\Roaming\Python\Python310\site-packages\transformers\modeling_utils.py:3353: UserWarning: Moving the following attributes in the config to the generation config: {'early_stopping': True, 'num_beams': 4, 'no_repeat_ngram_size': 3, 'forced_bos_token_id': 0}. You are seeing this warning because you've set generation parameters in the model config, as opposed to in the generation config.
  warnings.warn(


Saving trained model...
Calculating Post-training Metrics...


Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Post-training Metrics and Predictions saved!
Error in callback <bound method _WandbInit._pause_backend of <wandb.sdk.wandb_init._WandbInit object at 0x000001B033D763B0>> (for post_run_cell), with arguments args (<ExecutionResult object at 1b033d76b30, execution_count=3 error_before_exec=None error_in_exec=None info=<ExecutionInfo object at 1b033d76c80, raw_cell="import pandas as pd
import numpy as np
import torc.." store_history=True silent=False shell_futures=True cell_id=vscode-notebook-cell:/c%3A/Users/CSE%20RGUKT/Desktop/Stance/vast_filtered_im/Stance_Updated_2_im.ipynb#X15sZmlsZQ%3D%3D> result=None>,),kwargs {}:


TypeError: _WandbInit._pause_backend() takes 1 positional argument but 2 were given

**KeyBart**

In [ ]:
import pandas as pd
import numpy as np
import torch
from transformers import BartTokenizer, BartForConditionalGeneration, Trainer, TrainingArguments
from datasets import Dataset
from sklearn.metrics import f1_score
from rouge_score import rouge_scorer
import nltk
from bert_score import score as bert_score
from collections import Counter
import ot  # For WMD optimal transport

nltk.download('wordnet')
nltk.download('omw-1.4')
from nltk.translate.meteor_score import meteor_score

# Set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

def load_data(file_path):
    df = pd.read_csv(file_path)
    return df[['tweet', 'GT Target']]

def preprocess_data(df):
    df['tweet'] = df['tweet'].str.strip().str.lower()
    return df

def to_dataset(df):
    return Dataset.from_pandas(df)

def calculate_kqwr(prediction, reference):
    pred_words = set(prediction.lower().split())
    ref_words = set(reference.lower().split())
    if not pred_words or not ref_words:
        return 0.0
    overlap = len(pred_words & ref_words)
    total_unique = len(pred_words | ref_words)
    return overlap / total_unique if total_unique > 0 else 0.0

def calculate_td_tc(predictions, references):
    td = sum(1 for pred, ref in zip(predictions, references) if pred == ref)
    tc = len(predictions)
    return td / tc if tc > 0 else 0.0

def get_bert_embeddings(texts, tokenizer, model):
    if not texts:
        return np.zeros((1, 768))
    inputs = tokenizer(texts, return_tensors="pt", padding=True, truncation=True, max_length=512).to(device)
    with torch.no_grad():
        outputs = model(**inputs)
    return outputs.last_hidden_state.cpu().numpy()

def calculate_metrics(model, tokenizer, dataset, sample_size=100):
    model.eval()
    sample = dataset.select(range(min(sample_size, len(dataset))))
    predictions, references = [], []

    bert_tokenizer = BartTokenizer.from_pretrained('bloomz/keybart')  # Adjust checkpoint as needed
    bert_model = BartForConditionalGeneration.from_pretrained('bloomz/keybart').to(device)

    with torch.no_grad():
        for example in sample:
            input_text = example['tweet']
            inputs = tokenizer(input_text, return_tensors="pt", max_length=512, truncation=True).to(device)
            outputs = model.generate(**inputs, max_length=128, num_beams=4, early_stopping=True)
            pred = tokenizer.decode(outputs[0], skip_special_tokens=True).strip()
            predictions.append(pred)
            references.append(example['GT Target'])
            print(f"Predicted: {pred},Ground Truth : {example['GT Target']}")

    f1 = f1_score(references, predictions, average='weighted')
    scorer = rouge_scorer.RougeScorer(['rouge1', 'rougeL'], use_stemmer=True)
    rouge_scores = [scorer.score(ref, pred) for ref, pred in zip(references, predictions)]
    rouge1 = np.mean([s['rouge1'].fmeasure for s in rouge_scores])
    rougeL = np.mean([s['rougeL'].fmeasure for s in rouge_scores])
    meteor_scores = [meteor_score([ref.split()], pred.split()) for ref, pred in zip(references, predictions)]
    meteor = np.mean(meteor_scores)
    P, R, F1 = bert_score(predictions, references, lang="en", verbose=False)
    bertscore_f1 = F1.mean().item()

    kqwr_scores = [calculate_kqwr(pred, ref) for pred, ref in zip(predictions, references)]
    kqwr = np.mean(kqwr_scores)
    td_tc = calculate_td_tc(predictions, references)

    return {
        "F1": f1,
        "ROUGE-1": rouge1,
        "ROUGE-L": rougeL,
        "METEOR": meteor,
        "BERTScore": bertscore_f1,
        "KqWR": kqwr,
        "td/tc": td_tc,
    }

def tokenize_data(example, tokenizer):
    input_text = example['tweet']
    target_text = example['GT Target']
    inputs = tokenizer(input_text, max_length=512, truncation=True, padding="max_length", return_tensors="pt")
    targets = tokenizer(target_text, max_length=128, truncation=True, padding="max_length", return_tensors="pt")
    inputs["labels"] = targets["input_ids"]
    return inputs

def train_model(model, tokenizer, train_dataset):
    training_args = TrainingArguments(
        output_dir="./keybart_trained",
        num_train_epochs=3,
        per_device_train_batch_size=8,
        save_steps=500,
        save_total_limit=2,
        logging_steps=100,
    )
    trainer = Trainer(model=model, args=training_args, train_dataset=train_dataset)
    trainer.train()
    return model

def main(file_path):
    df = load_data(file_path)
    df = preprocess_data(df)
    dataset = to_dataset(df)

    tokenizer = BartTokenizer.from_pretrained("bloomz/keybart")  # Replace with actual KeyBART checkpoint
    model = BartForConditionalGeneration.from_pretrained("bloomz/keybart").to(device)

    print("Calculating Pre-training Metrics...")
    pre_metrics = calculate_metrics(model, tokenizer, dataset)
    print("Pre-training Metrics:", pre_metrics)

    train_size = min(1000, len(dataset))
    train_dataset = dataset.select(range(train_size))
    tokenized_train_dataset = train_dataset.map(lambda x: tokenize_data(x, tokenizer), batched=False)

    print("Training KeyBART...")
    trained_model = train_model(model, tokenizer, tokenized_train_dataset)

    print("Calculating Post-training Metrics...")
    post_metrics = calculate_metrics(trained_model, tokenizer, dataset.select(range(1000,1100)))
    print("Post-training Metrics:", post_metrics)

if __name__ == "__main__":
    main("/content/randomized_output.csv")

In [ ]:
import pandas as pd
import numpy as np
import torch
from transformers import T5Tokenizer, T5ForConditionalGeneration, BertTokenizer, BertModel
from datasets import Dataset
from sklearn.metrics import f1_score
from rouge_score import rouge_scorer
import nltk
from bert_score import score as bert_score
from collections import Counter
import ot

nltk.download('wordnet')
nltk.download('omw-1.4')
from nltk.translate.meteor_score import meteor_score


# Set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

def load_data(file_path):
    df = pd.read_csv(file_path)
    return df[['post', 'new_topic']]

def preprocess_data(df):
    df['post'] = df['post'].str.strip().str.lower()
    return df

def to_dataset(df):
    return Dataset.from_pandas(df)

def generate_keyphrases(model, tokenizer, text, num_keyphrases=5):
    # Modify input to instruct T5 to generate keyphrases
    input_text = f"Generate {num_keyphrases} keyphrases from the following text: {text}"
    inputs = tokenizer(input_text, return_tensors="pt", max_length=512, truncation=True).to(device)
    outputs = model.generate(**inputs, max_length=128, num_return_sequences=num_keyphrases, do_sample=True, top_k=50)
    keyphrases = [tokenizer.decode(output, skip_special_tokens=True).strip() for output in outputs]
    return keyphrases

def calculate_bertscore(predictions, reference):
    P, R, F1 = bert_score(predictions, [reference] * len(predictions), lang="en", verbose=False)
    return F1.numpy()  # Return F1 scores as numpy array

def select_best_keyphrase(keyphrases, reference):
    bert_scores = calculate_bertscore(keyphrases, reference)
    best_idx = np.argmax(bert_scores)
    return keyphrases[best_idx], bert_scores[best_idx]

def main(file_path):
    df = load_data(file_path)
    df = preprocess_data(df)
    dataset = to_dataset(df)

    tokenizer = T5Tokenizer.from_pretrained("t5-base")
    model = T5ForConditionalGeneration.from_pretrained("t5-base").to(device)

    # Select a sample for demonstration (e.g., first row)
    sample = dataset[0]
    post_text = sample['post']
    target_text = sample['new_topic']

    print(f"\nOriginal Post: {post_text}")
    print(f"Target (new_topic): {target_text}")

    # Generate 5 keyphrases using pre-trained T5
    print("\nGenerating 5 keyphrases with pre-trained T5...")
    keyphrases = generate_keyphrases(model, tokenizer, post_text, num_keyphrases=5)
    for i, kp in enumerate(keyphrases, 1):
        print(f"Keyphrase {i}: {kp}")

    # Calculate BERTScore and select best keyphrase
    print("\nCalculating BERTScore for each keyphrase against target...")
    best_keyphrase, best_score = select_best_keyphrase(keyphrases, target_text)
    bert_scores = calculate_bertscore(keyphrases, target_text)
    for i, (kp, score) in enumerate(zip(keyphrases, bert_scores), 1):
        print(f"Keyphrase {i}: {kp} | BERTScore F1: {score:.4f}")

    print(f"\nBest Keyphrase: {best_keyphrase}")
    print(f"Best BERTScore F1: {best_score:.4f}")

if __name__ == "__main__":
    main("vast_filtered_ex.csv")

[nltk_data] Downloading package wordnet to C:\Users\CSE
[nltk_data]     RGUKT\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to C:\Users\CSE
[nltk_data]     RGUKT\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


ValueError: unable to parse C:\Users\CSE RGUKT\.moverscore\vocab.txt as a URL or as a local path